<h2> What Does It Take To Be An Expert at Python </h2>

Notebook based on the https://www.youtube.com/watch?v=7lmCu8wz8ro 

In [ ]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    def __len__(self):
        return len(self.coeffs)

<h3> Summary </h3>

Top-level function or top-level syntax --> corresponding \__ function \__

x + y   --> \__add\__ <br/>
init x  --> \__init\__<br/>
repr(x) --> \__repr\__<br/>
len(x)  --> \__len__\__<br/>

These functions are called **Data Models** 
https://docs.python.org/3/reference/datamodel.html

In [ ]:
p1 = Polynomial(1,2,3)
p2 = Polynomial(3,4,3)

Python repr for the representation of the class Polynomial Object


In [ ]:
p1

Add function 
(Take note of zip() fucntion)

In [ ]:
p1 + p2

In [ ]:
len(p1)


# Some Analysis



In [ ]:
# library.py
class Base:
    def foo(self):
        return 'foo'


In [ ]:
# user.py
# from library import Base

assert hasattr(Base, 'foo'), "You broke it, you fool!"

class Derived(Base):
    def bar(self):
        return self.foo()

If **class Base** the **foo** function doesn't exist (could be named incorrectly or smething), <br/>
**assert hasattr** allows **class Derived** to check if the function exists before using it. 

NB: change foo name to something else and check where user.py fails. 

In [ ]:
#libray.py

class Base:
    def foo(self):
        return self.bar()

In [ ]:
#user.py
# from libray import Base

class Derived(Base):
    def bar(self):
        return 'bar'

How do we ensure that from class Base, the function foo returns a function called bar from the Derived class? Meaning we need to make sure that class Derived implements function foo. 

Let's do some analysis of something

In [ ]:
def _():
    class Base:
        pass

In [ ]:
from dis import dis

In [ ]:
dis(_)

LOAD_BUILD_CLASS suggests that there should be __ function __ that loads to build a class

# Buid_Class

In [ ]:
# library.py

class Base:
    def foo(self):
        return self.bar()

# Original build class for Base
old_bc = __build_class__

# define our own build class with a function
def my_bc(*a, **kw):
    print('my buildclass->', a, kw)
    return old_bc(*a, **kw)

# import __build_class__ from builtins
# and assign it to my_bc (my own built class)
import builtins
builtins.__build_class__ = my_bc

The above hijacks the build class process and interject our own imformation or build information, checks etc...

In [ ]:
# user.py
# from library import Base

class Derived:
    def bar(self):
        return 'bar'

The complete **Base** class would look like this: 
Using the original passed arguments

In [ ]:
# libray.py

class Base:
    def foo(self):
        return self.bar()
    
old_bc = __build_class__

def my_bc(fun, name, base=None, **kw):
    if base is Base:
        print('check if bar method is defined')
    if base is not None:
        return old_bc(fun, name, base, **kw)
    return old_bc(fun, name, **kw)

import builtins
builtins.__build_class__ = my_bc

The above solves the problem of checking if the class that will use class Base
implements the bar function. 

*This is one way of solving this problem, but there exists other ways that are commonly used to tackled this.* 

# Meta Classes

In [ ]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        print('BaseMeta.__new__', cls, name, bases, body)
        if not 'bar' in body:
            raise TypeError("bad user class")
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

In [ ]:
# user.py
# from library import Base

class Derived:
    def bar(self):
        return 'bar'

Another way to solve this problem is 

## sub_classes

In [1]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        print('BaseMeta.__new__', cls, name, bases, body)
        if name != 'Base' and not 'bar' in body:
            raise TypeError("bad user class")
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    
    def __init_subclass__(*a, **kw):
        print('init_subclass', a, kw)
        return super().__init_subclass__(*a, **kw)

BaseMeta.__new__ <class '__main__.BaseMeta'> Base () {'__module__': '__main__', '__qualname__': 'Base', 'foo': <function Base.foo at 0x109efbe18>, '__init_subclass__': <function Base.__init_subclass__ at 0x109efbea0>, '__classcell__': <cell at 0x109efc588: empty>}


In [2]:
help(Base.__init_subclass__)

Help on method __init_subclass__ in module __main__:

__init_subclass__(*a, **kw) method of __main__.BaseMeta instance
    This method is called when a class is subclassed.
    
    The default implementation does nothing. It may be
    overridden to extend subclasses.

